# Extraindo Entidades Nomeadas

## SpaCy

In [1]:
import spacy
from spacy import displacy

### Modelos pré-treinados

In [2]:
'''BAIXANDO O MODELO DA LÍNGUA PORTUGUESA'''
!python -m spacy download pt

2022-08-15 15:08:29.842265: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 5.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [3]:
'''CLASSIFICANDO CONTEÚDO ATRAVÉS DO MODELO'''

text = 'Donald Trump, portador do cpf 064.468.404-62, chegou aos EUA no dia 01/06/2019. Donald Trump visitou Barack Obama e George Bush. Além disso, ele esteve em diversos estados do país pilotando uma ferrari.'

my_model = spacy.load('pt')
document = my_model(text)

print('Original Sentence: %s\n\n' % (text))
    
displacy.render(document, jupyter=True, style='ent')

OSError: ignored

### Criando seu próprio modelo

In [ ]:
import random
def train_spacy(data,iterations, base_model=None):
    TRAIN_DATA = data
    if base_model:
      nlp = spacy.load(base_model)
      print("Loaded model '%s'" % nlp)
    else:
      nlp = spacy.blank('NER_NESS')      
      print('Created blank model')

    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],         # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,       # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
    return nlp

In [ ]:
TRAIN_DATA = [('Em 15/07/1988 nasceu essa linda criança', {'entities': [(3, 13, 'Data')]}),
              ('Rafael Mello, portador do cpf 064.468.404-62, chegou aos EUA', {'entities': [(0, 12, 'Pessoa'),(30, 44, 'CPF')]}),
              ('Data de prisão: 10/01/2018', {'entities': [(16, 26, 'Data')]}),
              ('No dia 01/02/2016 foi decretada a setença', {'entities': [(7, 17, 'Data')]}),
              ('A data da festa foi 07/05/2019', {'entities': [(20, 30, 'Data')]}),
              ('João, portador do cpf 123.456.789-65, foi encontrado', {'entities': [(0, 4, 'PER'), (18, 32, 'CPF')]}),
              ('O cpf 025.412.876-99 pertence a Maria', {'entities': [(6, 20, 'CPF')]})
             ]
           
my_model = train_spacy(TRAIN_DATA, 20)
document = my_model(text)

print('Original Sentence: %s\n' % (text))
displacy.render(document, jupyter=True, style='ent')

Se você estiver usando um modelo existente, certifique-se de misturar exemplos de outros tipos de entidade que o spaCy reconheceu corretamente antes. Caso contrário, seu modelo pode aprender o novo tipo, mas “esquecer” o que ele sabia anteriormente. Isso também é conhecido como o problema do “esquecimento catastrófico”.

In [ ]:
my_model2 = train_spacy(TRAIN_DATA, 20, 'pt')
document = my_model2(text)

print('Original Sentence: %s\n' % (text))
displacy.render(document, jupyter=True, style='ent')

## Adicionando padrões (dicionário)

O SpaCy permite que você defina seu próprio dicionário, utilizando ou não funções linguísticas (https://spacy.io/usage/rule-based-matching)

In [ ]:
from spacy.matcher import Matcher
from spacy.tokens import Span

matcher = Matcher(my_model.vocab)
matcher.add("CARRO", None, [{"LOWER": "ferrari"}])

doc = my_model(text)
matches = matcher(doc)

for match_id, start, end in matches:
    # create a new Span for each match and use the match_id (ANIMAL) as the label
    span = Span(doc, start, end, label=match_id)
    doc.ents = list(doc.ents) + [span]  # add span to doc.ents

displacy.render(doc, jupyter=True, style='ent')

## FLAIR
Utilizaremos inicialmente o Flair Framework para nossa tarefa de NER. É uma biblioteca que implementa o estado da arte em NLP atualmente, além de permitir o uso de modelos pré-treinados de forma bastante prática e simples. Ele utiliza o Pytorch como base para definição da sua arquitetura de redes neurais. 

### Instalando

In [ ]:
!pip install flair

### Testando

In [ ]:
#import commands for flair NER
from flair.data import Sentence
from flair.models import SequenceTagger

### Modelos Pré-treinados (EN)

Você pode ver a lista completa de modelos pré-treinados aqui: https://github.com/flairNLP/flair 

In [ ]:
#Load NER Model
tagger = SequenceTagger.load('ner')

In [ ]:
#Sample text to run NER
text = 'Jackson is placed in Microsoft located in Redmond'

#passing text to sentence
sentence = Sentence(text)

# Run NER on sentence to identify Entities
tagger.predict(sentence)

# print the entities with below command
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
print(sentence.to_tagged_string())

Testando outra sentença

In [ ]:
#Sample text
text1 = 'Redmond is coming to New York city'

#passing text to sentence
sentence = Sentence(text1)

# Run NER on sentence to identify Entities
tagger.predict(sentence)

# print the entities with below command
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
print(sentence.to_tagged_string())

### Treinando modelo FLAIR em Portugues

O treinamento de modelos do FLAIR é bem custoso, e normalmente obtém melhores resultados utilizando embeddings pré-treinados associados a embeddings específicos para fases forward e backward do algoritmo. 

Até o momento não existem modelos prétreinados disponíveis diretamente no framework, mas algumas iniciativas já estão em andamento (https://github.com/jneto04/ner-pt)